In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import pickle

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = './logs'
writer = tf.summary.FileWriter(logs_path)

df = pd.read_csv("shuffled-full-set-hashed.csv", header=None)

In [9]:
embeddings = pickle.load(open('embeddings.pkl', 'rb'))
embeddings

In [10]:
encoder = pickle.load(open('encoder.pkl', 'rb'))
encoder

LabelEncoder()

In [28]:
encoder.classes_

array(['APPLICATION', 'BILL', 'BILL BINDER', 'BINDER',
       'CANCELLATION NOTICE', 'CHANGE ENDORSEMENT', 'DECLARATION',
       'DELETION OF INTEREST', 'EXPIRATION NOTICE',
       'INTENT TO CANCEL NOTICE', 'NON-RENEWAL NOTICE', 'POLICY CHANGE',
       'REINSTATEMENT NOTICE', 'RETURNED CHECK'], dtype=object)

In [12]:
embeddings.wv['be95012ebf2b']

array([ 2.2876782 , -1.6393709 ,  1.0918756 , -0.15285057, -0.49653086,
       -1.7998664 ,  0.15558626,  0.32405075, -2.5706725 , -0.5051178 ,
       -1.823094  , -0.92694944,  1.9547423 , -3.9266665 ,  0.37842792,
       -1.6999105 , -0.83885455,  0.40298578,  0.33187824, -3.5896904 ,
        0.48616686,  2.1415029 , -0.40368897,  0.3654191 ,  0.31358495,
        1.6145602 , -3.2305148 , -0.36465952,  0.42663455,  1.222179  ,
       -3.7389407 ,  0.24845056, -2.0642395 , -1.1470802 , -1.9136102 ,
        2.5557392 , -0.8684581 , -1.5084149 ,  3.0187624 ,  0.6685917 ,
        0.58107966, -0.12966427,  1.018735  , -2.0941586 , -2.8666434 ,
        1.8626314 , -0.41905296,  0.05545424,  1.1254636 ,  0.07317962,
       -2.2134936 , -0.53294307, -3.2471688 , -3.270237  , -2.1740358 ,
       -1.787981  ,  0.14939575, -1.7104746 ,  1.0531877 , -1.7822953 ,
       -0.6566029 ,  0.19079603, -2.55527   , -0.34516266,  0.8153599 ,
       -2.491402  , -3.6351275 ,  1.4192287 ,  2.1528904 , -1.15

In [30]:
len(encoder.classes_)

14

In [31]:
len(embeddings.wv['be95012ebf2b'])

100

In [33]:
vocab_size = 100

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 5
output_classes = len(encoder.classes_) # 14

# number of units in RNN cell
n_hidden = 512

# # tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, output_classes])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}


In [34]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)


W0918 15:15:34.866117 140736158872448 deprecation.py:323] From <ipython-input-34-273144a7492a>:12: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0918 15:15:34.867493 140736158872448 deprecation.py:323] From <ipython-input-34-273144a7492a>:12: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0918 15:15:34.870103 140736158872448 deprecation.py:323] From <ipython-input-34-273144a7492a>:15: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent t

In [35]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()


W0918 15:16:00.430596 140736158872448 deprecation.py:323] From <ipython-input-35-a7db46f13d04>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0918 15:16:00.974018 140736158872448 deprecation.py:506] From /Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")
